In [1]:
collection_name = "chap_03"
connection_string = "http://localhost:6333"

In [2]:
from langchain_huggingface import HuggingFaceEmbeddings

# Define and load an embedding model, nomic it's optmized for italian
embeddings_model = HuggingFaceEmbeddings(model_name="intfloat/multilingual-e5-large")

In [3]:
from langchain_qdrant import QdrantVectorStore
from qdrant_client import QdrantClient

client = QdrantClient(
    url=connection_string,
)

# Retrieve embedding
qdrant = QdrantVectorStore(
    client=client,
    embedding=embeddings_model,
    collection_name=collection_name,

)

In [4]:
# Get retrieved
retriever = qdrant.as_retriever()

In [5]:
from langchain_ollama import OllamaLLM

# Import model
llm = OllamaLLM(model='gemma3')

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

# Prompt ch genera 5 versioni diverse della domanda
perspective_prompt = ChatPromptTemplate.from_template("""You are an AI language
    model assistant. Your task is to generate five different versions of the
    given user question to retrieve relevant documents from a vector database.
    By generating multiple perspectives on the user question, your goal is to
    help the user overcome some of the limitations of the distance-based
    similarity search. Provide these alternative questions separated by
    newlines. Original question: {question}""")

# Divide la risposta estrando le 5 versioni della domanda
def parse_queries_output(message):
    return message.split('\n')

# Crea un chain per il query gen
query_gen = perspective_prompt | llm | parse_queries_output

In [19]:
def get_unique_union(document_lists):
    deduped_docs = {
        doc.page_content: doc
        for sublist in document_lists for doc in sublist
    }
    return list(deduped_docs.values())

# Prende tutti i documenti che sono stati trovati dalle 5 query e forma un unica lista
retrieval_chain = query_gen | retriever.batch | get_unique_union

In [20]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate([
    ("system", """
     You are a helpful assistant answer the question based only on the following context: {context}.
     Answer in Italian. Always add in the anser the bible chapter and verses.
     Useful informations:
     1 cubit = 45,72 cm
     """),
    ("user", "Question: {question}")
])

In [15]:
from langchain_core.runnables import chain

@chain
def qa(question):
    docs = retrieval_chain.invoke(question)
    formatted = prompt.invoke({"context":docs, "question":question})
    return llm.invoke(formatted)


In [21]:
await qa.ainvoke("dimensioni dell'arca in centimetri")

Here are five alternative versions of the question "dimensioni dell'arca in centimetri" designed to broaden the search and potentially retrieve more relevant documents:

Size of the Ark in centimeters
Ark dimensions in centimeters
Measuring the Noah's Ark - dimensions
Noah's Ark size - cm
What is the length, width, and height of the Ark in centimeters?


"L'arca doveva essere lunga 300 cubiti, larga 50 cubiti e alta 30 cubiti.\n\nCalcoliamo le dimensioni in centimetri:\n\n*   Lunghezza: 300 cubiti * 45,72 cm/cubito = 13716 cm\n*   Larghezza: 50 cubiti * 45,72 cm/cubito = 2286 cm\n*   Altezza: 30 cubiti * 45,72 cm/cubito = 1371,6 cm\n\nQuindi, le dimensioni dell'arca erano: 13716 cm di lunghezza, 2286 cm di larghezza e 1371,6 cm di altezza. (Genesi 6:15)"